# HW4 Problem:  Classifying Music Genres

This problem asks you to classify music into genres that include:  Blues, Classical, Jazz, Metal, Pop, Rock.

Column 192 in the dataset is the "GENRE" attribute used for classification.
The 191 columns before this are numeric features of music clips.
<blockquote>
A database of 60 music performers has been prepared for the competition. The material is divided into six categories: classical music, jazz, blues, pop, rock and heavy metal. For each of the performers 15-20 music pieces have been collected. All music pieces are partitioned into 20 segments and parameterized. The descriptors used in parametrization also those formulated within the MPEG-7 standard, are only listed here since they have already been thoroughly reviewed and explained in many studies.  <br /><br />The feature vector consists of 191 parameters, the first 127 parameters are based on the MPEG-7 standard, the remaining ones are cepstral coefficients descriptors and time-related dedicated parameters:<br /><br />a) parameter 1: Temporal Centroid, <br />b) parameter 2: Spectral Centroid average value, <br />c) parameter 3: Spectral Centroid variance, <br />d) parameters 4-37: Audio Spectrum Envelope (ASE)  average values in 34 frequency bands<br />e) parameter 38: ASE average value (averaged for all frequency bands)<br />f) parameters 39-72: ASE variance values in 34 frequency bands<br />g) parameter 73: averaged ASE variance parameters<br />h) parameters 74,75: Audio Spectrum Centroid -- average and variance values<br />i) parameters 76,77: Audio Spectrum Spread -- average and variance values<br />j) parameters 78-101: Spectral Flatness Measure (SFM) average values for 24 frequency bands<br />k) parameter 102: SFM average value (averaged for all frequency bands)<br />l) parameters 103-126: Spectral Flatness Measure (SFM) variance values for 24 frequency bands<br />m) parameter 127: averaged SFM variance parameters<br />n) parameters 128-147: 20 first mel cepstral coefficients average values <br />o) parameters 148-167: the same as 128-147<br />p) parameters 168-191: dedicated parameters in time domain based of the analysis of the distribution of the envelope in relation to the rms value.<br />
</blockquote>

The results of a contest for building classifiers for this dataset are reported in:
<blockquote>
http://duch.mimuw.edu.pl/~mwojnars/papers/ismis-2011-contest.pdf
</blockquote>
This paper offers some ideas about models to consider.

#  The Goal

In this assignment you are to generate the genre predictions you can for a set of test data:
<ul><li>
Given the file <tt>MusicGenres.csv</tt>, develop a classifier that is as accurate as possible.
</li><li>
Use your classifier to predict genre classifications for each row of data in <tt>MusicFeatures.csv</tt>.
</li><li>
Put your predictions in a .csv file called  <tt>HW4_predictions.csv</tt> and upload it to CCLE.
</li></ul>

## Step 1: download your data, using your UID

<blockquote>

Download the music data at:
<br/>
http://datamining.cs.ucla.edu/cs249/hw4/music/___PUT_YOUR_UID_HERE___.zip

<br/>
<br/>
<i>For example, if your UID is  123456789, download the file</i>
    http://datamining.cs.ucla.edu/cs249/hw4/music/123456789.zip
    
</blockquote>
    
This zip file has two csv data files:  a training set and a test set.

## Step 2: construct a model from <tt>training_set.csv</tt>

Using the <tt>training_set.csv</tt> data, construct a classifier.

<br/>
<b>YOU CAN USE ANY ENVIRONMENT YOU LIKE TO BUILD A CLASSIFIER.</b>
Please construct the most accurate models you can.

<hr style="border-width:20px;">

## Step 3: generate predictions from <tt>test_set.csv</tt>
    
The rows of file <tt>test_set.csv</tt> have input features for a number of music clips.
Using your classifer, produce class predictions for each of them.

<br/>
Put one predicted class name per line in a CSV file <tt>HW4_Music_Predictions.csv</tt>.
This file should also have the header line "<tt>GENRE</tt>".

<br/>
<i>Your score on this problem will be the accuracy of these predictions.</i>
<br/>

<hr style="border-width:20px;">

## Step 4: upload <tt>HW4_Music_Predictions.csv</tt> and your notebook to CCLE

Finally, go to CCLE and upload:
<ul><li>
your output CSV file <tt>HW4_Music_Predictions.csv</tt>
</li><li>
your notebook file <tt>HW4_Music_Genres.ipynb</tt>
</li></ul>

We are not planning to run any of the uploaded notebooks.
However, your notebook should have the commands you used in developing your models ---
in order to show your work.
As announced, all assignment grading in this course will be automated,
and the notebook is needed in order to check results of the grading program.

In [13]:
################################################################################################
## Load all the libraries
################################################################################################

library(data.table)
library(caret)
library(kernlab)
library(glmnet)
library(gbm)
library(nnet)
library(class)
library(randomForest)
library(e1071)

In [14]:
################################################################################################
## Load all the features
################################################################################################

options(warn=-1)

# STEP 1 : download your data, using 204567818
# Reading in training and testing data
main.training.set = fread("204567818/HW4_music_data/training_set.csv", data.table = TRUE)
main.testing.set = fread("204567818/HW4_music_data/test_set.csv", data.table = TRUE)

In [15]:
################################################################################################
## Split data
################################################################################################

main.training.set=main.training.set[sample(nrow(main.training.set)),]
main.training.data = main.training.set
main.training.data$GENRE = NULL

split.idx = createDataPartition(main.training.set$GENRE, p = .7,
                                list = TRUE,
                                times = 1)

training.set = main.training.set[split.idx$Resample1, ]
testing.set = main.training.set[-split.idx$Resample1, ]

print(dim(training.set))
print(dim(testing.set))

training.data = training.set
training.data$GENRE = NULL

testing.data= testing.set
testing.data$GENRE = NULL

[1] 7872  192
[1] 3373  192


In [ ]:
################################################################################################
## Normalize data
################################################################################################

procValues = preProcess(main.training.data, method = c("center", "scale"))

normalize.main.training.data = predict(procValues, main.training.data)
normalize.main.testing.set = predict(procValues, main.testing.set)

normalize.main.training.set = data.frame(cbind(normalize.main.training.data, main.training.set$GENRE))
colnames(normalize.main.training.set) = c(colnames(normalize.main.training.data), "GENRE")


procValues = preProcess(training.data, method = c("center", "scale"))
normalize.training.data = predict(procValues, training.data)
normalize.testing.data = predict(procValues, testing.data)

normalize.training.set = data.frame(cbind(normalize.training.data, training.set$GENRE))
colnames(normalize.training.set) = c(colnames(normalize.training.data), "GENRE")
normalize.testing.set =  data.frame(cbind(normalize.testing.data, testing.set$GENRE))
colnames(normalize.testing.set) = c(colnames(normalize.testing.data), "GENRE")

control = trainControl(method="repeatedcv", number=10, repeats=3)

In [16]:
################################################################################################
## Accuracy Function ## 
################################################################################################

calc.accuracy = function(predicted, actual){
  return(length(which(predicted == actual)) / length(actual))
}

In [17]:
################################################################################################
## MODEL BUILDING ## 
################################################################################################
## Multinomial Logistic Regression
################################################################################################
nnet.fit = multinom(GENRE ~. , data = training.set, MaxNWts = 10000, trControl=control)
nnet.fit.n = multinom(GENRE ~. , data = normalize.training.set, MaxNWts = 10000, trControl=control)

predicted.genre.softmax = predict(nnet.fit, newdata = testing.set)
predicted.genre.softmax.n = predict(nnet.fit.n, newdata = normalize.testing.set)

cat("\nMultinomial Logistic Regression Accuracy \n")
cat("Without Normalize ", calc.accuracy(predicted.genre.softmax, testing.set$GENRE), "\n")
cat("With Normalize ", calc.accuracy(predicted.genre.softmax.n, testing.set$GENRE), "\n")

# weights:  193 (192 variable)
initial  value 5456.454605 
iter  10 value 2623.976198
iter  20 value 1408.331519
iter  30 value 704.742557
iter  40 value 478.476200
iter  50 value 414.142411
iter  60 value 367.135578
iter  70 value 332.156219
iter  80 value 317.311693
iter  90 value 304.776714
iter 100 value 298.001506
final  value 298.001506 
stopped after 100 iterations
# weights:  193 (192 variable)
initial  value 5456.454605 
iter  10 value 698.738767
iter  20 value 606.730573
iter  30 value 512.802478
iter  40 value 405.323934
iter  50 value 369.796713
iter  60 value 352.671366
iter  70 value 339.724473
iter  80 value 324.292829
iter  90 value 278.601225
iter 100 value 222.137110
final  value 222.137110 
stopped after 100 iterations
Multinomial Logistic Regression Accuracy 
Without Normalize  0.9807293 
With Normalize  0.9807293 


In [18]:
################################################################################################
## Lasso Logistic Regression
################################################################################################

glmnet.fitted.model = glmnet(x = as.matrix(training.data),y = training.set$GENRE,
                             family="multinomial", lambda=0.04)
glmnet.fitted.model.n = glmnet(x = as.matrix(normalize.training.data),y = training.set$GENRE,
                               family="multinomial", lambda=0.04)

predicted.genre.lasso = predict(glmnet.fitted.model, newx=as.matrix(testing.data), type="class")
predicted.genre.lasso.n = predict(glmnet.fitted.model.n, newx=as.matrix(normalize.testing.data), type="class")

cat("Lasso Logistic Regression Accuracy\n")
cat("Without Normalize ", calc.accuracy(predicted.genre.lasso, testing.set$GENRE), "\n")
cat("With Normalize ", calc.accuracy(predicted.genre.lasso.n, testing.set$GENRE), "\n")

Lasso Logistic Regression Accuracy
Without Normalize  0.9552327 
With Normalize  0.9552327 


In [19]:
################################################################################################
## Support Vector Machines
################################################################################################

svm.model = svm(factor(GENRE) ~ ., data = training.set, cv.folds=5, trControl=control)
svm.model.n = svm(factor(GENRE) ~ ., data = normalize.training.set, cv.folds=5, trControl=control)

predicted.genre.svm.n = predict(svm.model.n, normalize.testing.set)
predicted.genre.svm = predict(svm.model, testing.data)

cat("Support Vector Machine Accuracy\n")
cat("Without Normalize ", calc.accuracy(predicted.genre.svm, testing.set$GENRE), "\n")
cat("With Normalize ", calc.accuracy(predicted.genre.svm.n, testing.set$GENRE), "\n")

Support Vector Machine Accuracy
Without Normalize  0.9875482 
With Normalize  0.9875482 


In [20]:
################################################################################################
## Random Forest
################################################################################################

rf.model = randomForest(factor(GENRE) ~ .,data = training.set, cv.folds=5, trControl=control)
rf.model.n = randomForest(factor(GENRE) ~ .,data = normalize.training.set, cv.folds=5, trControl=control)

predicted.genre.rf = predict(rf.model, newdata = as.matrix(testing.data), type="response", predict.all = FALSE)
predicted.genre.rf.n = predict(rf.model.n, newdata = as.matrix(normalize.testing.data), type="response", predict.all = FALSE)

cat("Random Forest Accuracy\n")
cat("Without Normalize ", calc.accuracy(predicted.genre.rf, testing.set$GENRE), "\n")
cat("With Normalize ", calc.accuracy(predicted.genre.rf.n, testing.set$GENRE), "\n")

Random Forest Accuracy
Without Normalize  0.9857693 
With Normalize  0.9851764 


In [21]:
################################################################################################
## Emsemble Classifier
################################################################################################
# Used RANDOM FOREST, SVM & SOFTMAX & LASSO
testing.output = data.frame(predicted.genre.rf, predicted.genre.svm, predicted.genre.softmax,predicted.genre.lasso)   
testing.output.n = data.frame(predicted.genre.rf.n, predicted.genre.svm.n, predicted.genre.softmax.n,predicted.genre.lasso.n)

to = apply(testing.output,1,function(x) names(which.max(table(x))))
vo = apply(testing.output.n,1,function(x) names(which.max(table(x))))

cat("Ensemble Classifier Accuracy \n")
cat("Without Normalize ", calc.accuracy(to, testing.set$GENRE), "\n")
cat("With Normalize ", calc.accuracy(vo, testing.set$GENRE), "\n")

Ensemble Classifier Accuracy 
Without Normalize  0.979247 
With Normalize  0.9801364 


In [22]:
################################################################################################
## Main Model Building 
################################################################################################

# NORMALIZED DATA
nnet.fit.main = multinom(GENRE ~. , data = normalize.main.training.set, MaxNWts = 20000)
svm.model.main = svm(factor(GENRE) ~ ., data = normalize.main.training.set, cv.folds = 10)
rf.model.main = randomForest(factor(GENRE) ~ .,data = normalize.main.training.set, cv.folds = 10)
glmnet.fitted.main.n = glmnet(x = as.matrix(normalize.main.training.data),y = main.training.set$GENRE,
                               family="multinomial", lambda=0.04)

predicted.genre.softmax.main = predict(nnet.fit.main, newdata = as.matrix(normalize.main.testing.set))
predicted.genre.svm.main = predict(svm.model.main, newdata = as.matrix(normalize.main.testing.set))
predicted.genre.rf.main = predict(rf.model.main, newdata = as.matrix(normalize.main.testing.set), type="response", predict.all = FALSE)
predicted.genre.lasso.n = predict(glmnet.fitted.model.n, newx=as.matrix(normalize.main.testing.set), type="class")

normalize.ensemble.classifier = data.frame(predicted.genre.rf.main, predicted.genre.svm.main, predicted.genre.softmax.main
                                           ,predicted.genre.lasso.n) # Used RANDOM FOREST, SVM & SOFTMAX & Lasso
normalize.ensemble.classifier.output = matrix(apply(normalize.ensemble.classifier, 1, function(x) names(which.max(table(x)))), ncol = 1)
colnames(normalize.ensemble.classifier.output) = c("GENRE")

# weights:  193 (192 variable)
initial  value 7794.440045 
iter  10 value 1268.623991
iter  20 value 987.832283
iter  30 value 778.805429
iter  40 value 548.149157
iter  50 value 464.447413
iter  60 value 430.333977
iter  70 value 417.124245
iter  80 value 402.555677
iter  90 value 393.608463
iter 100 value 376.257190
final  value 376.257190 
stopped after 100 iterations


In [24]:
# Write output into CSV File
write.table(normalize.ensemble.classifier.output, file = "HW4_Music_Predictions.csv", append = FALSE, quote = TRUE, sep = " ",
            eol = "\n", na = "NA", dec = ".", row.names = FALSE,
            col.names = TRUE, qmethod = c("escape", "double"),
            fileEncoding = "")